In [1]:
# Install required libraries
!pip install torch
!pip install transformers
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00


In [2]:
# Load and Move the FLAN-T5 Model and its tokenizer to the GPU
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Check if the tokenizer works by testing it on a sample text
sample_text = "I am testing"
tokens = tokenizer(sample_text)
print(tokens)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

{'input_ids': [27, 183, 2505, 1], 'attention_mask': [1, 1, 1, 1]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# Load the dataset
from datasets import load_dataset

# Load the original dataset and reduce its size
ds = load_dataset("rajpurkar/squad")
train_ds = ds['train'].shuffle(seed=42).select(range(10000))  # Adjust the range if needed


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [4]:
# Define the preprocess function
def preprocess_function(examples):
    # Tokenize the context and question
    inputs = tokenizer(
        examples['question'],
        examples['context'],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )

    # Initialize a list to hold the first answers
    first_answers = []
    for answer in examples['answers']:
        if len(answer['text']) > 0:  # Check if the text list is not empty
            first_answers.append(answer['text'][0])  # Append the first answer
        else:
            first_answers.append("")  # Append an empty string if no answers

    # Tokenize the answers
    labels = tokenizer(
        first_answers,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )['input_ids']

    inputs['labels'] = labels
    return inputs

# Preprocess the dataset using the reduced training set
tokenized_ds = train_ds.map(preprocess_function, batched=True)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [5]:
# Split the tokenized dataset into training and validation sets (80% train, 20% validation)
train_test_split = tokenized_ds.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']


In [6]:
from transformers import TrainingArguments

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Save model at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,  # Load the best checkpoint after training
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
from transformers import Trainer

# Initialize the trainer with model, dataset, and training arguments
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Start training
trainer.train()


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.020200,0.011522
2,0.015500,0.011328
3,0.014800,0.011338


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=4800, training_loss=0.5776627769072851, metrics={'train_runtime': 4445.2324, 'train_samples_per_second': 5.399, 'train_steps_per_second': 1.08, 'total_flos': 1.6434176458752e+16, 'train_loss': 0.5776627769072851, 'epoch': 3.0})

In [8]:
# Function to generate an answer based on a context and question
def generate_answer(context, question):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, padding=True).to("cuda")
    outputs = model.generate(inputs.input_ids)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Example context and question
context = "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It was constructed between 1887 and 1889 as the entrance arch for the 1889 World's Fair."
question = "When was the Eiffel Tower constructed?"

# Generate the answer
answer = generate_answer(context, question)
print("Question:", question)
print("Answer:", answer)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Question: When was the Eiffel Tower constructed?
Answer: between 1887 and 1889


In [9]:
# Evaluate the model on the validation set
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.011327519081532955, 'eval_runtime': 108.1114, 'eval_samples_per_second': 18.499, 'eval_steps_per_second': 3.7, 'epoch': 3.0}


In [10]:
# Save the trained model and tokenizer
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")


('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/spiece.model',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [13]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [14]:
# Push the model and tokenizer to Hugging Face with a unique name
model.push_to_hub("tootooba/flan-t5-qa-study-assistant")
tokenizer.push_to_hub("tootooba/flan-t5-qa-study-assistant")


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tootooba/flan-t5-qa-study-assistant/commit/df22670a9b48003ce9d05ce6de87a8030a174e15', commit_message='Upload tokenizer', commit_description='', oid='df22670a9b48003ce9d05ce6de87a8030a174e15', pr_url=None, pr_revision=None, pr_num=None)